# LayerNorm and RMSNorm Analysis

本notebook实现

1. pytorch自动计算LayerNorm梯度
2. 手动计算LayerNorm梯度
3. 实现RMSNorm
4. 手动计算RMSNorm梯度
5. 分析RMSNorm的不变性
6. LayerNorm和RMSNorm对比梯度大小

# config

In [1]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
seed = 42
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)​ # if you have CUDA

In [2]:
batch_size = 1
seq_len = 4
d_model = 8
eps=1e-12
N = batch_size * seq_len * d_model

In [3]:
src = torch.randn(batch_size, seq_len, d_model) # 输入
trg = torch.randn(batch_size, seq_len, d_model) # 输入
# proj = nn.Linear(d_model, d_model)

In [4]:
def mse_loss(x, y):
    # bs, seq_len, d = y.size()
    # N = bs*seq_len*d
    return (((x - y) ** 2) * 0.5).mean()
loss = mse_loss(src, trg)
print(loss)

tensor(1.1714)

# pytorch自动计算LayerNorm梯度

In [5]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        # layernorm作用在(-1) 最后一维进行归一化
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        out_mean_var = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out_mean_var + self.beta # feature level
        return mean, var, out_mean_var, out

class ToyModel(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(ToyModel, self).__init__()
        self.w1 = nn.Linear(d_model,d_model)
        self.ln = LayerNorm(d_model, eps=1e-12)
        
    def forward(self, x):   
        w1_x = self.w1(x)
        # w1_x.retain_grad()
        mean, var, out_mean_var, out = self.ln(w1_x)
        return mean, var, out_mean_var, out, w1_x 

In [6]:
w_ln_model = ToyModel(d_model)
mean, var, out_mean_var, y, w1_x = w_ln_model(src)
w1_x.retain_grad()
y.retain_grad()
loss = mse_loss(y, trg)
loss.backward()

In [7]:
print(w_ln_model.ln.gamma.grad)
print(w_ln_model.ln.beta.grad)
print(w_ln_model.w1.weight.shape) # 只保留一小部分用于验证
print(w_ln_model.w1.weight.grad[:3,:3]) # 只保留一小部分用于验证

print(w1_x.grad.shape) # 只保留一小部分用于验证
print(w1_x.grad[:3,:3]) # 只保留一小部分用于验证

print(y.grad.shape)
print(y.grad[:3,:3])

tensor([ 0.4787,  0.2260,  0.0547,  0.0974,  0.0267, -0.0207,  0.0022,  0.1098])

tensor([ 0.3311,  0.0179, -0.0236, -0.0566,  0.0232, -0.1172, -0.0123, -0.0101])

torch.Size([8, 8])

tensor([[ 0.1667,  0.5198,  0.0378],
        [ 0.2474, -0.1831,  0.1003],
        [ 0.0218,  0.0555,  0.0255]])

torch.Size([1, 4, 8])

tensor([[[ 0.0815,  0.0531,  0.0025, -0.1298, -0.0359,  0.0203, -0.0325,
           0.0409],
         [ 0.1814, -0.1567,  0.0053,  0.0644,  0.0517, -0.2121,  0.0317,
           0.0344],
         [ 0.0266,  0.0172, -0.0120, -0.0292,  0.0100, -0.0542,  0.0223,
           0.0193]]])

torch.Size([1, 4, 8])

tensor([[[ 0.0812, -0.0326,  0.0262, -0.0622,  0.0215, -0.0200, -0.0123,
           0.0400],
         [ 0.0874, -0.0401, -0.0070,  0.0464,  0.0086, -0.0306, -0.0038,
           0.0302],
         [ 0.0529,  0.0789, -0.0281, -0.0434,  0.0130, -0.0085,  0.0001,
          -0.0082]]])

# 手动计算LayerNorm梯度

根据公式我们有：

de/dy * dy/dx * dx/d_src

## de/dy

In [8]:
de_dy = (y-trg) / N # /N is mean
print(de_dy[:3,:3])
print(y.grad[:3,:3])

tensor([[[ 0.0812, -0.0326,  0.0262, -0.0622,  0.0215, -0.0200, -0.0123,
           0.0400],
         [ 0.0874, -0.0401, -0.0070,  0.0464,  0.0086, -0.0306, -0.0038,
           0.0302],
         [ 0.0529,  0.0789, -0.0281, -0.0434,  0.0130, -0.0085,  0.0001,
          -0.0082]]], grad_fn=<SliceBackward0>)

tensor([[[ 0.0812, -0.0326,  0.0262, -0.0622,  0.0215, -0.0200, -0.0123,
           0.0400],
         [ 0.0874, -0.0401, -0.0070,  0.0464,  0.0086, -0.0306, -0.0038,
           0.0302],
         [ 0.0529,  0.0789, -0.0281, -0.0434,  0.0130, -0.0085,  0.0001,
          -0.0082]]])

先求出不同的gamma和beta

In [9]:
dgamma = out_mean_var 
dgamma = (dgamma * de_dy).sum(1)
print(dgamma)
print(w_ln_model.ln.gamma.grad)

tensor([[ 0.4787,  0.2260,  0.0547,  0.0974,  0.0267, -0.0207,  0.0022,  0.1098]],
       grad_fn=<SumBackward1>)

tensor([ 0.4787,  0.2260,  0.0547,  0.0974,  0.0267, -0.0207,  0.0022,  0.1098])

In [10]:
dbeta = de_dy.sum(1)
print(dbeta)
print(w_ln_model.ln.beta.grad)

tensor([[ 0.3311,  0.0179, -0.0236, -0.0566,  0.0232, -0.1172, -0.0123, -0.0101]],
       grad_fn=<SumBackward1>)

tensor([ 0.3311,  0.0179, -0.0236, -0.0566,  0.0232, -0.1172, -0.0123, -0.0101])

## dy/dx

In [11]:
x_mean = w1_x - mean
x_var = var # 方差
x_std_var = torch.sqrt(x_var + eps)
print(x_std_var)

tensor([[[0.4516],
         [0.2954],
         [0.8131],
         [0.3634]]], grad_fn=<SqrtBackward0>)

In [12]:
I = torch.ones(d_model)
diag_I = I.diag()
print(I)
print(diag_I)

I = torch.ones(d_model, d_model)

left = diag_I - 1/d_model * I
print(left)

tensor([1., 1., 1., 1., 1., 1., 1., 1.])

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

tensor([[ 0.8750, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250],
        [-0.1250,  0.8750, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250],
        [-0.1250, -0.1250,  0.8750, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250],
        [-0.1250, -0.1250, -0.1250,  0.8750, -0.1250, -0.1250, -0.1250, -0.1250],
        [-0.1250, -0.1250, -0.1250, -0.1250,  0.8750, -0.1250, -0.1250, -0.1250],
        [-0.1250, -0.1250, -0.1250, -0.1250, -0.1250,  0.8750, -0.1250, -0.1250],
        [-0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250,  0.8750, -0.1250],
        [-0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250, -0.1250,  0.8750]])

In [13]:
# first we get token-level gradient 8x8, 
# 一个token有8维，ln前后的雅可比有grad_x_ln 8x8
# dx'1/dx1 dx'1/dx2 , ..  dx'1/dx8
# ...
# dx'8/dx1 dx'8/dx2 , ..  dx'8/dx8

# 四个token的梯度为de_dy = 4x8
# 一个token反向的梯度为  de_dy[1,: ]= 1x8
# 一个token的ln的梯度为de_dy @ grad_x_ln =  1x8

grad_x_ln = torch.zeros_like(w1_x)
for i in range(batch_size):
    for j in range(seq_len):
        d_ln = (left/x_std_var[i,j,0] -  (x_mean[i,j,:].outer(x_mean[i,j,:]))/(x_std_var[i,j,0]**3 * d_model)) * w_ln_model.ln.gamma
        grad_x_ln[i,j,:] = de_dy[i, j, :] @ d_ln
print(de_dy.shape)
print(grad_x_ln)

torch.Size([1, 4, 8])

tensor([[[ 0.0815,  0.0531,  0.0025, -0.1298, -0.0359,  0.0203, -0.0325,
           0.0409],
         [ 0.1814, -0.1567,  0.0053,  0.0644,  0.0517, -0.2121,  0.0317,
           0.0344],
         [ 0.0266,  0.0172, -0.0120, -0.0292,  0.0100, -0.0542,  0.0223,
           0.0193],
         [ 0.0800, -0.0008,  0.0318, -0.0645,  0.0398, -0.1455,  0.1418,
          -0.0827]]], grad_fn=<CopySlices>)

In [14]:
print(w1_x.grad)

tensor([[[ 0.0815,  0.0531,  0.0025, -0.1298, -0.0359,  0.0203, -0.0325,
           0.0409],
         [ 0.1814, -0.1567,  0.0053,  0.0644,  0.0517, -0.2121,  0.0317,
           0.0344],
         [ 0.0266,  0.0172, -0.0120, -0.0292,  0.0100, -0.0542,  0.0223,
           0.0193],
         [ 0.0800, -0.0008,  0.0318, -0.0645,  0.0398, -0.1455,  0.1418,
          -0.0827]]])

## dx/d_w

In [15]:
dx_d_w = grad_x_ln.transpose(1,2) @ src
print(dx_d_w)
print(w_ln_model.w1.weight.grad)
ln_w_gradient = dx_d_w

tensor([[[ 1.6670e-01,  5.1977e-01,  3.7791e-02, -3.0767e-01, -1.1550e-01,
          -1.7000e-01, -1.4177e-01,  1.2115e-01],
         [ 2.4742e-01, -1.8313e-01,  1.0033e-01,  1.1461e-01,  1.3727e-01,
           4.0570e-02,  1.3214e-01, -1.7655e-01],
         [ 2.1801e-02,  5.5484e-02,  2.5527e-02,  2.4503e-02, -4.2252e-04,
          -1.7609e-02, -2.1741e-02,  7.2077e-03],
         [-4.2904e-01, -1.6580e-01, -1.6705e-01,  8.3870e-02, -9.7860e-02,
           8.9999e-02, -5.1096e-02,  1.5282e-01],
         [-4.0714e-02,  8.1779e-02, -3.3314e-02,  6.0696e-02, -7.8804e-02,
           2.7918e-02, -4.0178e-02,  1.4813e-01],
         [-7.6512e-02, -4.9953e-01,  3.9672e-02,  3.7114e-02,  2.4294e-01,
           2.9126e-02,  1.5540e-01, -4.1043e-01],
         [ 1.3156e-01,  1.8411e-01,  3.3760e-02,  2.2313e-01, -9.4864e-02,
           5.2419e-02, -4.0747e-02,  2.3575e-01],
         [-2.1214e-02,  7.3143e-03, -3.6716e-02, -2.3625e-01,  7.2391e-03,
          -5.2420e-02,  7.9920e-03, -7.8090e-02]]],
       grad_fn=<UnsafeViewBackward0>)

tensor([[ 1.6670e-01,  5.1977e-01,  3.7791e-02, -3.0767e-01, -1.1550e-01,
         -1.7000e-01, -1.4177e-01,  1.2115e-01],
        [ 2.4742e-01, -1.8313e-01,  1.0033e-01,  1.1461e-01,  1.3727e-01,
          4.0570e-02,  1.3214e-01, -1.7655e-01],
        [ 2.1802e-02,  5.5484e-02,  2.5527e-02,  2.4503e-02, -4.2251e-04,
         -1.7609e-02, -2.1741e-02,  7.2077e-03],
        [-4.2904e-01, -1.6580e-01, -1.6705e-01,  8.3870e-02, -9.7860e-02,
          8.9999e-02, -5.1096e-02,  1.5282e-01],
        [-4.0714e-02,  8.1779e-02, -3.3314e-02,  6.0696e-02, -7.8804e-02,
          2.7918e-02, -4.0178e-02,  1.4813e-01],
        [-7.6512e-02, -4.9953e-01,  3.9672e-02,  3.7114e-02,  2.4294e-01,
          2.9126e-02,  1.5540e-01, -4.1043e-01],
        [ 1.3156e-01,  1.8411e-01,  3.3760e-02,  2.2313e-01, -9.4864e-02,
          5.2419e-02, -4.0747e-02,  2.3575e-01],
        [-2.1214e-02,  7.3143e-03, -3.6716e-02, -2.3625e-01,  7.2391e-03,
         -5.2420e-02,  7.9920e-03, -7.8090e-02]])

综上我们验证成功手写layernorm梯度

# 实现RMSNorm

In [16]:
class RMSNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(RMSNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.eps = eps

    def forward(self, x):
        mean = (x**2).mean(-1, keepdim=True)
        out_mean = x / torch.sqrt(mean + self.eps) # root mean square
        out = self.gamma * out_mean 
        return mean, out_mean, out

class ToyModelRMS(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(ToyModelRMS, self).__init__()
        self.w1 = nn.Linear(d_model,d_model)
        self.rms = RMSNorm(d_model, eps=1e-12)
        
    def forward(self, x):   
        w1_x = self.w1(x)
        mean, out_mean, out = self.rms(w1_x)
        return mean, out_mean, out, w1_x 

In [17]:
w_rms_model = ToyModelRMS(d_model)
# 保持当前的
w_rms_model.w1.weight.data = w_ln_model.w1.weight.data.clone()
w_rms_model.w1.zero_grad()

mean, out_mean, y, w1_x = w_rms_model(src)
w1_x.retain_grad()
y.retain_grad()
loss = mse_loss(y, trg)
loss.backward()

In [18]:
print(w_rms_model.rms.gamma.grad)
# print(w_rms_model.rms.beta.grad)
print(w_rms_model.w1.weight.shape) 
print(w_rms_model.w1.weight.grad) 

print(w1_x.grad.shape)
print(w1_x.grad[:3,:3])

print(y.grad.shape)
print(y.grad[:3,:3])

tensor([0.0090, 0.1799, 0.1195, 0.0944, 0.1186, 0.1652, 0.0641, 0.1910])

torch.Size([8, 8])

tensor([[ 0.1785,  0.4463,  0.0312, -0.2583, -0.1054, -0.1332, -0.1084,  0.1211],
        [ 0.2390, -0.1757,  0.0805,  0.1387,  0.1076,  0.0636,  0.1308, -0.1186],
        [ 0.0068,  0.1388,  0.0067, -0.0973, -0.0277, -0.0577, -0.0504,  0.0181],
        [-0.3286,  0.0547, -0.1306, -0.0523, -0.1256,  0.0052, -0.1057,  0.1683],
        [-0.0776,  0.2686, -0.0682, -0.1873, -0.1366, -0.0611, -0.1087,  0.1728],
        [ 0.0765, -0.5129,  0.1067,  0.2436,  0.2654,  0.0804,  0.1916, -0.3770],
        [ 0.0713,  0.1807, -0.0084,  0.0439, -0.0976,  0.0152, -0.0436,  0.1862],
        [ 0.0300,  0.0969, -0.0243, -0.2211, -0.0226, -0.0581, -0.0121, -0.0188]])

torch.Size([1, 4, 8])

tensor([[[ 0.0736,  0.0394,  0.0185, -0.0828, -0.0043,  0.0168, -0.0124,
           0.0440],
         [ 0.1523, -0.1435,  0.0549,  0.1278,  0.1585, -0.2778,  0.0665,
           0.0573],
         [ 0.0371,  0.0294, -0.0044, -0.0226,  0.0202, -0.0478,  0.0316,
           0.0280]]])

torch.Size([1, 4, 8])

tensor([[[ 0.0633,  0.0042,  0.0442, -0.0350,  0.0506, -0.0104,  0.0060,
           0.0501],
         [ 0.0206, -0.0461, -0.0106,  0.0572,  0.0410, -0.0782, -0.0225,
           0.0133],
         [ 0.0292,  0.0797, -0.0297, -0.0437,  0.0244, -0.0255, -0.0060,
          -0.0172]]])

# 手动计算RMSNorm梯度

根据公式我们有：

de/dy * dy/dx * dx/d_src

## de/dy

In [19]:
de_dy = (y-trg) / N # /N is mean
print(de_dy[:3,:3])
print(y.grad[:3,:3])

tensor([[[ 0.0633,  0.0042,  0.0442, -0.0350,  0.0506, -0.0104,  0.0060,
           0.0501],
         [ 0.0206, -0.0461, -0.0106,  0.0572,  0.0410, -0.0782, -0.0225,
           0.0133],
         [ 0.0292,  0.0797, -0.0297, -0.0437,  0.0244, -0.0255, -0.0060,
          -0.0172]]], grad_fn=<SliceBackward0>)

tensor([[[ 0.0633,  0.0042,  0.0442, -0.0350,  0.0506, -0.0104,  0.0060,
           0.0501],
         [ 0.0206, -0.0461, -0.0106,  0.0572,  0.0410, -0.0782, -0.0225,
           0.0133],
         [ 0.0292,  0.0797, -0.0297, -0.0437,  0.0244, -0.0255, -0.0060,
          -0.0172]]])

## de/dy * dy/dgamma

In [20]:
dgamma = out_mean
dgamma = (dgamma * de_dy).sum(1)
print(dgamma)
print(w_rms_model.rms.gamma.grad)

tensor([[0.0090, 0.1799, 0.1195, 0.0944, 0.1186, 0.1652, 0.0641, 0.1910]],
       grad_fn=<SumBackward1>)

tensor([0.0090, 0.1799, 0.1195, 0.0944, 0.1186, 0.1652, 0.0641, 0.1910])

## dy/dx

In [21]:
x = w1_x
x_rms = torch.sqrt(mean + eps)
print(x_rms)

tensor([[[0.6207],
         [0.2539],
         [0.7705],
         [0.6786]]], grad_fn=<SqrtBackward0>)

In [22]:
# first we get token-level gradient 8x8, 
# 一个token有8维，ln前后的雅可比有grad_x_ln 8x8
# dx'1/dx1 dx'1/dx2 , ..  dx'1/dx8
# ...
# dx'8/dx1 dx'8/dx2 , ..  dx'8/dx8

# 四个token的梯度为de_dy = 4x8
# 一个token反向的梯度为  de_dy[1,: ]= 1x8
# 一个token的ln的梯度为de_dy[i,:] @ grad_x_ln[i,:] =  1x8
I_diag = torch.diag(torch.ones(d_model))

grad_x_rms = torch.zeros_like(w1_x)
for i in range(batch_size):
    for j in range(seq_len):
        d_rms = I_diag/x_rms[i,j,0] -  (w1_x[i,j,:].outer(w1_x[i,j,:]))/(x_rms[i,j,0]**3 * d_model) *  w_rms_model.rms.gamma
        grad_x_rms[i,j,:] = de_dy[i, j, :] @ d_rms 
print(de_dy.shape)
print(grad_x_rms )

torch.Size([1, 4, 8])

tensor([[[ 0.0736,  0.0394,  0.0185, -0.0828, -0.0043,  0.0168, -0.0124,
           0.0440],
         [ 0.1523, -0.1435,  0.0549,  0.1278,  0.1585, -0.2778,  0.0665,
           0.0573],
         [ 0.0371,  0.0294, -0.0044, -0.0226,  0.0202, -0.0478,  0.0316,
           0.0280],
         [ 0.0706,  0.0054,  0.0155, -0.0281,  0.0130, -0.0674,  0.0729,
          -0.0452]]], grad_fn=<CopySlices>)

In [23]:
print(w1_x.grad)

tensor([[[ 0.0736,  0.0394,  0.0185, -0.0828, -0.0043,  0.0168, -0.0124,
           0.0440],
         [ 0.1523, -0.1435,  0.0549,  0.1278,  0.1585, -0.2778,  0.0665,
           0.0573],
         [ 0.0371,  0.0294, -0.0044, -0.0226,  0.0202, -0.0478,  0.0316,
           0.0280],
         [ 0.0706,  0.0054,  0.0155, -0.0281,  0.0130, -0.0674,  0.0729,
          -0.0452]]])

## dx/dx_src

In [24]:
dx_d_w = grad_x_rms.transpose(1,2) @ src
print(dx_d_w)
print(w_rms_model.w1.weight.grad)

rms_w_gradient = dx_d_w

tensor([[[ 0.1785,  0.4463,  0.0312, -0.2583, -0.1054, -0.1332, -0.1084,
           0.1211],
         [ 0.2390, -0.1757,  0.0805,  0.1387,  0.1076,  0.0636,  0.1308,
          -0.1186],
         [ 0.0068,  0.1388,  0.0067, -0.0973, -0.0277, -0.0577, -0.0504,
           0.0181],
         [-0.3286,  0.0547, -0.1306, -0.0523, -0.1256,  0.0052, -0.1057,
           0.1683],
         [-0.0776,  0.2686, -0.0682, -0.1873, -0.1366, -0.0611, -0.1087,
           0.1728],
         [ 0.0765, -0.5129,  0.1067,  0.2436,  0.2654,  0.0804,  0.1916,
          -0.3770],
         [ 0.0713,  0.1807, -0.0084,  0.0439, -0.0976,  0.0152, -0.0436,
           0.1862],
         [ 0.0300,  0.0969, -0.0243, -0.2211, -0.0226, -0.0581, -0.0121,
          -0.0188]]], grad_fn=<UnsafeViewBackward0>)

tensor([[ 0.1785,  0.4463,  0.0312, -0.2583, -0.1054, -0.1332, -0.1084,  0.1211],
        [ 0.2390, -0.1757,  0.0805,  0.1387,  0.1076,  0.0636,  0.1308, -0.1186],
        [ 0.0068,  0.1388,  0.0067, -0.0973, -0.0277, -0.0577, -0.0504,  0.0181],
        [-0.3286,  0.0547, -0.1306, -0.0523, -0.1256,  0.0052, -0.1057,  0.1683],
        [-0.0776,  0.2686, -0.0682, -0.1873, -0.1366, -0.0611, -0.1087,  0.1728],
        [ 0.0765, -0.5129,  0.1067,  0.2436,  0.2654,  0.0804,  0.1916, -0.3770],
        [ 0.0713,  0.1807, -0.0084,  0.0439, -0.0976,  0.0152, -0.0436,  0.1862],
        [ 0.0300,  0.0969, -0.0243, -0.2211, -0.0226, -0.0581, -0.0121, -0.0188]])

# 分析RMSNorm的不变性

## 前向输入不变性

见RMS Paper的4.1 Invariance Analysis

尺度不变性

$$
RMS(\alpha x) = \alpha RMS(x)
$$

In [28]:
w_rms_model = ToyModelRMS(d_model, eps=1e-12)
x = w_rms_model.w1(src)

x_10 = x * 10
mean, out_mean, out,  = w_rms_model.rms(x_10)
print(mean)
print(out_mean[0,0,:])
print(out[0,0,:])
x_rms_10 = torch.sqrt(mean + eps)

mean, out_mean, out,  = w_rms_model.rms(x)
print(out_mean[0,0,:])
print(out[0,0,:])
x_rms = torch.sqrt(mean + eps)

print('rms forward is scalar invariant')

tensor([[[57.8723],
         [14.7380],
         [41.7154],
         [ 7.7856]]], grad_fn=<MeanBackward1>)

tensor([-1.0879,  1.4208,  0.4922, -1.3188, -1.1799,  0.1664,  0.1501,  1.1722],
       grad_fn=<SliceBackward0>)

tensor([-1.0879,  1.4208,  0.4922, -1.3188, -1.1799,  0.1664,  0.1501,  1.1722],
       grad_fn=<SliceBackward0>)

tensor([-1.0879,  1.4208,  0.4922, -1.3188, -1.1799,  0.1664,  0.1501,  1.1722],
       grad_fn=<SliceBackward0>)

tensor([-1.0879,  1.4208,  0.4922, -1.3188, -1.1799,  0.1664,  0.1501,  1.1722],
       grad_fn=<SliceBackward0>)

rms forward is scalar invariant

## 反向梯度不变性

见RMS Paper的4.2 Gradient Analysis

In [34]:
# 
I_diag = torch.diag(torch.ones(d_model))

# scalar 10
grad_x_rms_10 = torch.zeros_like(x)
for i in range(batch_size):
    for j in range(seq_len):
        d_rms = I_diag/x_rms_10[i,j,0] -  (x_10[i,j,:].outer(x_10[i,j,:]))/(x_rms_10[i,j,0]**3 * d_model)
        # d_rms = d_rms * 10 # x' = x * 10
        grad_x_rms_10[i,j,:] = de_dy[i, j, :] @ d_rms 
print(grad_x_rms_10 )


# scalar 1
grad_x_rms = torch.zeros_like(x)
for i in range(batch_size):
    for j in range(seq_len):
        d_rms = I_diag/x_rms[i,j,0] -  (x[i,j,:].outer(x[i,j,:]))/(x_rms[i,j,0]**3 * d_model) 
        grad_x_rms[i,j,:] = de_dy[i, j, :] @ d_rms
print(grad_x_rms )

# rms norm 梯度和输入成反比
print(grad_x_rms_10 / grad_x_rms )

tensor([[[ 0.0084,  0.0005,  0.0058, -0.0045,  0.0067, -0.0014,  0.0008,
           0.0065],
         [ 0.0039, -0.0115, -0.0058,  0.0131,  0.0121, -0.0205, -0.0059,
           0.0008],
         [ 0.0048,  0.0125, -0.0039, -0.0069,  0.0046, -0.0022,  0.0002,
          -0.0019],
         [ 0.0157, -0.0043, -0.0072, -0.0100, -0.0056, -0.0348,  0.0006,
          -0.0316]]], grad_fn=<CopySlices>)

tensor([[[ 0.0838,  0.0048,  0.0578, -0.0453,  0.0672, -0.0138,  0.0079,
           0.0653],
         [ 0.0389, -0.1152, -0.0576,  0.1310,  0.1206, -0.2052, -0.0585,
           0.0081],
         [ 0.0485,  0.1246, -0.0393, -0.0688,  0.0462, -0.0220,  0.0021,
          -0.0187],
         [ 0.1565, -0.0434, -0.0716, -0.0997, -0.0556, -0.3484,  0.0059,
          -0.3156]]], grad_fn=<CopySlices>)

tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
         [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
         [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
         [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000]]],
       grad_fn=<DivBackward0>)

# LayerNorm和RMSNorm对比梯度大小

In [35]:
print(rms_w_gradient.norm())
print(ln_w_gradient.norm())

tensor(1.2899, grad_fn=<LinalgVectorNormBackward0>)

tensor(1.3214, grad_fn=<LinalgVectorNormBackward0>)

# 去Center化作用

ref：https://spaces.ac.cn/archives/8620#%E7%9B%B4%E6%8E%A5%E6%A0%87%E5%87%86%E5%8C%96

“
一个直观的猜测是，center操作，类似于全连接层的bias项，储存到的是关于预训练任务的一种先验分布信息，而把这种先验分布信息直接储存在模型中，反而可能会导致模型的迁移能力下降。所以T5不仅去掉了Layer Normalization的center操作，它把每一层的bias项也都去掉了。
”

# Reference

LayerNorm 梯度推导： [Layer Normalization, and how to compute its Jacobian for Backpropagation?](https://neuralthreads.medium.com/layer-normalization-and-how-to-compute-its-jacobian-for-backpropagation-55a549d5936f)

RMSNorm Paper: [Root Mean Square Layer Normalization](https://arxiv.org/pdf/1910.07467)

感谢 @Julian Lou 给出修改意见。